In [1]:
from NACT.utils import *
from NACT import Scanpy_IO, ScanpyObj_IO, AttentionQuery
import numpy as np
import os
import scanpy as sc
from scGeneFit.functions import *
import seaborn as sns
from sklearn.metrics import f1_score, accuracy_score
from sklearn.neighbors import NearestCentroid, KNeighborsClassifier
import torch
from xgboost import XGBClassifier 
import matplotlib.pyplot as plt

%matplotlib inline
np.random.seed(0) 

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def get_logp1_variance(data:np.array)->float:
    return np.var(np.log(data + 1), axis=0).sum()

def explained_variance_ratio(full_dimension_data:np.array, train_data:np.array, test_data:np.array)->float:
    all_data = np.concatenate((train_data, test_data), axis=0)
    return (get_logp1_variance(all_data) / get_logp1_variance(full_dimension_data))

def performance(X_train, y_train, X_test, y_test, clf, scoring='weighted'):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return f1_score(y_test, y_pred, average=scoring)
    

In [4]:
# clf=NearestCentroid()
clf = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
# clf = XGBClassifier(n_estimators=50, max_depth=2, learning_rate=0.09, objective='binary:logistic', n_jobs=-1)

# Accuracy on All 5000 Genes

In [5]:
path_to_data = "/home/aheydari/data/NACT_Data/Supervised Benchmarking/GSE_ImmuneCSF_qc_hvg_anno_5k_raw_train_split.h5ad"
adata = sc.read(path_to_data)

In [6]:
adata_train = adata[adata.obs.split=="train"]
adata_test = adata[adata.obs.split=="test"]

labels_train  = adata_train.obs.cluster.to_numpy()
labels_test  = adata_test.obs.cluster.to_numpy()

In [7]:
start = time.time()
print("==> KNN classification for baseline:")
accuracy=performance(adata_train.X.todense(), labels_train.astype(int) ,adata_test.X.todense(), labels_test.astype(int), clf)
print(f"Weighted F1 for all genes (whole data 5000 markers): {accuracy}")
print(f"KNN on All 5000 Genes took {time.time() - start} seconds")

==> KNN classification for baseline:


/home/aheydari/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Weighted F1 for all genes (whole data 5000 markers): 0.7510437784766674
KNN on All 5000 Genes took 143.58326959609985 seconds


# Cell Ranger

In [8]:
adata = sc.read(path_to_data)
adata

AnnData object with n_obs × n_vars = 75540 × 5000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'barcodes', 'celltypes', 'condition', 'patient', 'percent_mito', 'RNA_snn_res.0.2', 'RNA_snn_res.0.4', 'general_celltypes', 'kmeans_11', 'encoded_celltypes', 'cluster', 'split'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable'
    uns: 'RNA_snn_res.0.2_colors', 'celltypes_colors', 'condition_colors', 'neighbors', 'patient_colors'
    obsm: 'X_harmony', 'X_pca', 'X_umap'
    varm: 'HARMONY', 'PCs'
    obsp: 'distances'

In [9]:
top_n = [5, 10, 20, 50, 100, 200, 300]
start = time.time()

for n in top_n:
    print(f"For {n} markers")
    sc.pp.highly_variable_genes(adata, n_top_genes=n, flavor='cell_ranger', inplace=True)
    hvg_adata = adata[:, adata.var.highly_variable]
    adata_train = hvg_adata[hvg_adata.obs.split=="train"]
    adata_test = hvg_adata[hvg_adata.obs.split=="test"]
    
    data_train = np.array(adata_train.X.todense())
    labels_train  = adata_train.obs.cluster.to_numpy()
    names_train = list(adata_train.obs.celltypes.to_numpy())

    data_test = np.array(adata_test.X.todense())
    labels_test  = adata_test.obs.cluster.to_numpy()
    names_test = list(adata_test.obs.celltypes.to_numpy())

    accuracy_markers=performance(data_train, labels_train, data_test, labels_test, clf)

    print(f"Weighted F1 for selected {n} markers) {accuracy_markers:.04f}")
    print(f"Variance Explained: {explained_variance_ratio(full_dimension_data=np.array(adata.X.todense()), train_data = data_train, test_data = data_test)*100}%")
    print()

print(f"Time : {time.time()-start} seconds")

For 5 markers


/home/aheydari/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Weighted F1 for selected 5 markers) 0.4636
Variance Explained: 0.16279711853712797%

For 10 markers


/home/aheydari/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Weighted F1 for selected 10 markers) 0.1668
Variance Explained: 0.5424338858574629%

For 20 markers


...

Weighted F1 for selected 100 markers) 0.8495
Variance Explained: 4.027225822210312%

For 200 markers


/home/aheydari/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Weighted F1 for selected 200 markers) 0.8747
Variance Explained: 6.781486421823502%

For 300 markers


/home/aheydari/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Weighted F1 for selected 300 markers) 0.8761
Variance Explained: 8.740483224391937%

Time : 137.8828558921814 seconds


# Seurat

In [10]:
adata = sc.read(path_to_data)
adata

AnnData object with n_obs × n_vars = 75540 × 5000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'barcodes', 'celltypes', 'condition', 'patient', 'percent_mito', 'RNA_snn_res.0.2', 'RNA_snn_res.0.4', 'general_celltypes', 'kmeans_11', 'encoded_celltypes', 'cluster', 'split'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable'
    uns: 'RNA_snn_res.0.2_colors', 'celltypes_colors', 'condition_colors', 'neighbors', 'patient_colors'
    obsm: 'X_harmony', 'X_pca', 'X_umap'
    varm: 'HARMONY', 'PCs'
    obsp: 'distances'

In [11]:
top_n = [5, 10, 20, 50, 100, 200, 300]
start = time.time()

for n in top_n:
    print(f"For {n} markers")
    sc.pp.highly_variable_genes(adata, n_top_genes=n, flavor='seurat', inplace=True)
    hvg_adata = adata[:, adata.var.highly_variable]
    adata_train = hvg_adata[hvg_adata.obs.split=="train"]
    adata_test = hvg_adata[hvg_adata.obs.split=="test"]
    
    data_train = np.array(adata_train.X.todense())
    labels_train  = adata_train.obs.cluster.to_numpy()
    names_train = list(adata_train.obs.celltypes.to_numpy())

    data_test = np.array(adata_test.X.todense())
    labels_test  = adata_test.obs.cluster.to_numpy()
    names_test = list(adata_test.obs.celltypes.to_numpy())

    accuracy_markers=performance(data_train, labels_train, data_test, labels_test, clf)

    print(f"Weighted F1 for selected {n} markers) {accuracy_markers:.04f}")
    print(f"Variance Explained: {explained_variance_ratio(full_dimension_data=np.array(adata.X.todense()), train_data = data_train, test_data = data_test)*100}%")
    print()



For 5 markers


/home/aheydari/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Weighted F1 for selected 5 markers) 0.3763
Variance Explained: 0.029429918504320085%

For 10 markers


/home/aheydari/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Weighted F1 for selected 10 markers) 0.3883
Variance Explained: 0.06597836618311703%

For 20 markers


...

Weighted F1 for selected 100 markers) 0.5940
Variance Explained: 0.607574824243784%

For 200 markers


/home/aheydari/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Weighted F1 for selected 200 markers) 0.7221
Variance Explained: 2.224314399063587%

For 300 markers


/home/aheydari/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Weighted F1 for selected 300 markers) 0.7786
Variance Explained: 3.57033908367157%



# NACT

In [12]:
adata = sc.read(path_to_data)

In [13]:
model_dict_mixed = torch.load("/home/aheydari/data/NACT_Trained_Models/New/Marker Gene Selection/NACT-Pojections + Attention-Imune CSF-1Heads-Bestmodel_Best.pth",
                        map_location=torch.device('cpu'))

nact_mixed = model_dict_mixed["Saved_Model"]

nact_mixed = nact_mixed.eval()
nact_mixed

NACT_ProjectionAttention(
  (attention): Linear(in_features=5000, out_features=5000, bias=True)
  (proj_block): Projection(
    (projection): Linear(in_features=5000, out_features=5000, bias=True)
    (output_dropout): Dropout(p=0.0, inplace=False)
    (normalization): LayerNorm((5000,), eps=1e-05, elementwise_affine=True)
  )
  (proj_block2): Projection(
    (projection): Linear(in_features=5000, out_features=5000, bias=True)
    (output_dropout): Dropout(p=0.0, inplace=False)
    (normalization): LayerNorm((5000,), eps=1e-05, elementwise_affine=True)
  )
  (pwff): PointWiseFeedForward(
    (conv1): Sequential(
      (0): Linear(in_features=5000, out_features=128, bias=True)
      (1): ReLU()
    )
    (conv2): Linear(in_features=128, out_features=5000, bias=True)
    (normalization): LayerNorm((5000,), eps=1e-05, elementwise_affine=True)
  )
  (nact_out_layer): Sequential(
    (0): Linear(in_features=5000, out_features=15, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
)

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if str(device) == "cuda":
    print('Using GPU (CUDA)')
else:
    print('Using CPU')
    
nact_mixed.device = 'cpu'

Using GPU (CUDA)


## Attention Analysis

In [15]:
aq_obj = AttentionQuery(adata, split_test=False)
att_adata, att_df= aq_obj.AssignAttention(model=nact_mixed, inplace=False, correct_preditions_only=False ,verbose=True, use_raw_X=False)

*Caution*: The method is running on the entire data. If this is not what you want, provide scanpy object
==> Calling forward:
    -> Making predictions
    -> Assigning attention weights globally: 
    -> Creating a [Cells x Attention Per Gene'] DataFrame
    -> Could not locate obs['gene_ids'] attribute. Defaulting to .var instead
    -> Returning the annData with the attention weights


In [16]:
top_n = [5, 10, 20, 50, 100, 200, 300][::-1]

for n in top_n:
    print(f"==> For n = {n}:")
    top_n = aq_obj.GetTopN(n=n, correct_preditions_only=False).index.tolist()
    att_adata_train = att_adata[att_adata.obs.split=="train"]
    att_adata_test = att_adata[att_adata.obs.split=="test"]
    att_data_train = np.array(att_adata_train.X.todense()) # att_adata_train.obsm['gene_scores']
    att_labels_train  = att_adata_train.obs.cluster.to_numpy()
    att_names_train = list(att_adata_train.obs.celltypes.to_numpy())
    att_data_test = np.array(att_adata_test.X.todense()) # att_adata_test.obsm['gene_scores'] #
    att_labels_test  = att_adata_test.obs.cluster.to_numpy()
    att_names_test = list(att_adata_test.obs.celltypes.to_numpy())
    # get the data for only the top n genes
    reduced_adata = att_adata[:,top_n]
    reduced_adata_train = reduced_adata[reduced_adata.obs.split=="train"]
    reduced_adata_test = reduced_adata[reduced_adata.obs.split=="test"]
    
    
    reduced_train = np.array(reduced_adata_train.X.todense()) #att_adata_train.obsm['gene_scores']
    reduced_labels_train  = reduced_adata_train.obs.cluster.to_numpy()
    reduced_names_train = list(reduced_adata_train.obs.celltypes.to_numpy())
    
    reduced_test = np.array(reduced_adata_test.X.todense()) #att_adata_test.obsm['gene_scores']
    reduced_labels_test  = reduced_adata_test.obs.cluster.to_numpy()
    reduced_names_test = list(reduced_adata_test.obs.celltypes.to_numpy())
    
    accuracy_markers=performance(reduced_train, reduced_labels_train.astype(int), reduced_test, reduced_labels_test.astype(int), clf)
    print(f"Weighted F1 for selected {n} markers {accuracy_markers:.04f}")
    print(f"Variance Explained: {explained_variance_ratio(full_dimension_data=np.array(adata.X.todense()), train_data = reduced_train, test_data = reduced_test)*100}%")
    print("---------------------------")

==> For n = 300:
==> Getting Top 300 genes for all cells in the original data
    -> Be cautious as this may not be cluster specific. If you want cluster specific, call GetTopN_PerClust()


/home/aheydari/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Weighted F1 for selected 300 markers 0.8510
Variance Explained: 30.411437153816223%
---------------------------
==> For n = 200:
==> Getting Top 200 genes for all cells in the original data
    -> Be cautious as this may not be cluster specific. If you want cluster specific, call GetTopN_PerClust()


/home/aheydari/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Weighted F1 for selected 200 markers 0.8619
Variance Explained: 20.30424177646637%
---------------------------
==> For n = 100:
==> Getting Top 100 genes for all cells in the original data
    -> Be cautious as this may not be cluster specific. If you want cluster specific, call GetTopN_PerClust()


...

Weighted F1 for selected 20 markers 0.7740
Variance Explained: 1.31781455129385%
---------------------------
==> For n = 10:
==> Getting Top 10 genes for all cells in the original data
    -> Be cautious as this may not be cluster specific. If you want cluster specific, call GetTopN_PerClust()


/home/aheydari/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Weighted F1 for selected 10 markers 0.6746
Variance Explained: 0.33470115158706903%
---------------------------
==> For n = 5:
==> Getting Top 5 genes for all cells in the original data
    -> Be cautious as this may not be cluster specific. If you want cluster specific, call GetTopN_PerClust()


/home/aheydari/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Weighted F1 for selected 5 markers 0.5132
Variance Explained: 0.12361943954601884%
---------------------------


# scGeneFit

In [17]:
adata = sc.read(path_to_data)

# separating train and test split
adata_train = adata[adata.obs.split=="train"]
adata_test = adata[adata.obs.split=="test"]

# getting numerical data for training and testing from the split ann data
data_train = np.array(adata_train.X.todense())
labels_train  = adata_train.obs.cluster.to_numpy()
names_train = list(adata_train.obs.celltypes.to_numpy())

data_test = np.array(adata_test.X.todense())
labels_test  = adata_test.obs.cluster.to_numpy()
names_test = list(adata_test.obs.celltypes.to_numpy())

# running scGeneFit for various number of desired top genes
top_n = [5, 10, 20, 50, 100, 200, 300]
all_markers = []
method='centers'
redundancy=0.001
epsilon = 0.01

for n in top_n:
    print(f"    -> For {n} markers")
    markers= get_markers(data_train[:,:], labels_train.astype(int), n, method=method, redundancy=redundancy, epsilon=epsilon)
    accuracy_markers=performance(data_train[:,markers], labels_train.astype(int), data_test[:,markers], labels_test.astype(int), clf)
    all_markers.append(markers)
    
    print(f"    -> Weighted F1 for selected {n} markers: {accuracy_markers:.04f}")
    print(f"Variance Explained: {explained_variance_ratio(full_dimension_data=np.array(adata.X.todense()), train_data = data_train[:,markers], test_data = data_test[:,markers])*100}%")

    print("----------------------")
    
print(">-< Done")

    -> For 5 markers
    -> Solving a linear program with 5000 variables and 11 constraints
    -> Time elapsed: 0.5711565017700195 seconds


/home/aheydari/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


    -> Weighted F1 for selected 5 markers: 0.4505
Variance Explained: 0.3116628620773554%
----------------------
    -> For 10 markers
    -> Solving a linear program with 5000 variables and 11 constraints
    -> Time elapsed: 0.5706417560577393 seconds


/home/aheydari/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


    -> Weighted F1 for selected 10 markers: 0.4378
Variance Explained: 0.45099626295268536%
----------------------
    -> For 20 markers
    -> Solving a linear program with 5000 variables and 11 constraints
    -> Time elapsed: 0.5491352081298828 seconds


...

    -> Weighted F1 for selected 100 markers: 0.7993
Variance Explained: 4.7122035175561905%
----------------------
    -> For 200 markers
    -> Solving a linear program with 5000 variables and 11 constraints
    -> Time elapsed: 0.5827343463897705 seconds


/home/aheydari/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


    -> Weighted F1 for selected 200 markers: 0.8211
Variance Explained: 7.882394641637802%
----------------------
    -> For 300 markers
    -> Solving a linear program with 5000 variables and 11 constraints
    -> Time elapsed: 0.5670645236968994 seconds


/home/aheydari/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


    -> Weighted F1 for selected 300 markers: 0.8539
Variance Explained: 11.326269805431366%
----------------------
>-< Done


# Random Features Selected

In [19]:
np.random.seed(0) 

adata = sc.read(path_to_data)

# separating train and test split
adata_train = adata[adata.obs.split=="train"]
adata_test = adata[adata.obs.split=="test"]

# getting numerical data for training and testing from the split ann data
data_train = np.array(adata_train.X.todense())
labels_train  = adata_train.obs.cluster.to_numpy()
names_train = list(adata_train.obs.celltypes.to_numpy())

data_test = np.array(adata_test.X.todense())
labels_test  = adata_test.obs.cluster.to_numpy()
names_test = list(adata_test.obs.celltypes.to_numpy())

# running scGeneFit for various number of desired top genes
top_n = [5, 10, 20, 50, 100, 200, 300]
all_markers = []

for n in top_n:
    print(f"    -> For {n} markers")
    random_markers= np.random.randint(low=0, high=data_train.shape[1], size=n)
    accuracy_markers=performance(data_train[:,random_markers], labels_train.astype(int), data_test[:,random_markers], labels_test.astype(int), clf)
    all_markers.append(markers)
    
    print(f"    -> Weighted F1 for selected {n} RANDOM markers: {accuracy_markers:.04f}")
    print(f"Variance Explained: {explained_variance_ratio(full_dimension_data=np.array(adata.X.todense()), train_data = data_train[:,random_markers], test_data = data_test[:,random_markers])*100}%")

    print("----------------------")
    
print(">-< Done")

    -> For 5 markers


/home/aheydari/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


    -> Weighted F1 for selected 5 RANDOM markers: 0.3777
Variance Explained: 0.0661732570733875%
----------------------
    -> For 10 markers


/home/aheydari/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


    -> Weighted F1 for selected 10 RANDOM markers: 0.4330
Variance Explained: 0.1626685610972345%
----------------------
    -> For 20 markers


...

    -> Weighted F1 for selected 100 RANDOM markers: 0.6177
Variance Explained: 2.1295979619026184%
----------------------
    -> For 200 markers


/home/aheydari/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


    -> Weighted F1 for selected 200 RANDOM markers: 0.6887
Variance Explained: 4.536724090576172%
----------------------
    -> For 300 markers


/home/aheydari/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


    -> Weighted F1 for selected 300 RANDOM markers: 0.6652
Variance Explained: 6.370954215526581%
----------------------
>-< Done
